In [ ]:
# Start writing code here...
import matplotlib.pyplot as plt
import torch

import fastai.vision.all as vision
from PIL import Image
import numpy as np
import torch
import seaborn as sns

from mean_classifier import MeanClassifier
from gradient_descent_classifier import GradientDescentClassifier

from gradient_descent_regressor import GradientDescentRegressor
import utils

X_train, X_test, y_train, y_test = utils.load_mnist()

In [ ]:
path = vision.untar_data(vision.URLs.MNIST_SAMPLE)

train_threes = (path/'train'/'3').ls().sorted()
train_threes = [utils.image_to_tensor(img) for img in train_threes]
train_threes = torch.stack(train_threes).float() / 255

train_sevens = (path/'train'/'7').ls().sorted()
train_sevens = [utils.image_to_tensor(img) for img in train_sevens]
train_sevens = torch.stack(train_sevens).float() / 255


test_threes = (path/'valid'/'3').ls().sorted()
test_threes = [utils.image_to_tensor(img) for img in test_threes]
test_threes = torch.stack(test_threes).float() / 255
test_sevens = (path/'valid'/'7').ls().sorted()
test_sevens = [utils.image_to_tensor(img) for img in test_sevens]
test_sevens = torch.stack(test_sevens).float() / 255

X_test = torch.cat((test_sevens, test_threes)).view(-1, 28*28)
X_train = torch.cat((train_sevens, train_threes)).view(-1, 28*28)

y_train = torch.from_numpy(np.array([0] * len(train_sevens) + [1] * len(train_threes)))
y_test = torch.from_numpy(np.array([0] * len(test_sevens) + [1] * len(test_threes)))  

In [ ]:
import torch
import torch.nn.functional as F


class GradientDescentClassifier:
    def __init__(self, lr=1, epochs=200, seed=None):
        self.lr = lr
        self.epochs = epochs
        self.weights = None
        self.bias = None
        self.seed = seed

    def fit(self, X, y):
        self.initialise(X)
        for i in range(self.epochs):
            self.epoch(X, y)
        return self

    def initialise(self, X):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        self.weights = torch.randn(X.shape[1]).requires_grad_()
        self.bias = torch.randn(1).requires_grad_()

    def proba(self, X):
        return ((X @ self.weights) + self.bias).sigmoid()


    def epoch(self, X, y):
        epoch_loss = self.loss(X, y)
        epoch_loss.backward()
        self.weights.data -= self.weights.grad.data * self.lr
        self.bias.data -= self.bias.grad.data * self.lr
        self.weights.grad = None
        self.bias.grad = None
        # print(epoch_loss, self.score(X, y))

    def loss(self, X, y_true):
        probs = self.proba(X)
        return torch.where(y_true == 1, 1-probs, probs).mean()

    def predict(self, X):
        probs = self.proba(X)
        return probs.round()

    def score(self, X, y_true):
        preds = self.predict(X)
        return (preds == y_true).float().mean()


clf = GradientDescentClassifier(seed=1337)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

tensor(0.9553)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d3089e79-0e42-40a4-9bfd-5171fd4144aa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>